In [ ]:
#Libraries


import os
import cv2
import numpy as np
from math import hypot

#show image
def display_image(current_image):
    cv2.imshow("Image",current_image)
    cv2.waitKey(0)
    
    
def grab_cut(raw_image):
    
    #paramteres
    number_of_interation = 5
    channel_mask = np.zeros((raw_image.shape[0], raw_image.shape[1]), np.uint8)
    region_of_interest = (50, 50, 450, 290)
    background_model = np.zeros((1,65),np.float64)
    foreground_model = np.zeros((1,65),np.float64)
    
    #Cutting the image
    cv2.grabCut(raw_image, channel_mask, region_of_interest, background_model, foreground_model, number_of_interation, cv2.GC_INIT_WITH_RECT)
    
    #normalizing the image
    normalization_mask = np.where((channel_mask == 2)|(channel_mask == 0), 0, 1).astype('uint8')
    
    #remove background
    removed_background_image = raw_image * (normalization_mask[:,:,np.newaxis])
    return removed_background_image

#grayscale image
def grayscale(raw_image):
    
    return cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY)


#Bitwise invertion images
def invert_grayscale(grayscaled_image):
    
    return cv2.bitwise_not(grayscaled_image)

#noise reduction using blackhat method
def black_hat(grayscaled_image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.morphologyEx(inverted_grayscaled_image,cv2.MORPH_BLACKHAT,kernel)

#pixel with high white intensity are averaged
def median_blur(raw_image):
    image_without_reflection = cv2.medianBlur(removed_refection, 5)
    cv2.equalizeHist(image_without_reflection)
    return image_without_reflection


#detect edges by pixel intensity
def canny_edge(raw_image):
    region_of_interest = cv2.bitwise_not(raw_image)
    retval, thresholded_image = cv2.threshold(region_of_interest, 50, 255,cv2.THRESH_BINARY_INV)
    return cv2.Canny(thresholded_image, 200, 100)canny 


#detecting circles using hough circles method
def hough_circles(edged_image):
    
    return cv2.HoughCircles(edged_image, cv2.HOUGH_GRADIENT, 1, 20, param1 = 200, param2 = 20, minRadius = 0)

#draws circles as per the co-ordinates given
def draw_circle(circles, raw_image):
    inner_circle = np.uint16(np.around(circles[0][0])).tolist()
    cv2.circle(raw_image, (inner_circle[0], inner_circle[1]), inner_circle[2], (0, 255, 0), 1)
    return inner_circle


#get correct circle and crop image
def crop_image(x_coordinate, y_coordinate, inner_circle, raw_image):
    for j in range(x_coordinate):
        for k in range(y_coordinate):
            if hypot(k - inner_circle[0], j - inner_circle[1]) >= inner_circle[2]:
                raw_image[j, k] = 0
                

current_image = cv2.imread("eye.jpg", 1)
#show input image
display_image(current_image)

#first step grabcut
background_removed_image = grab_cut(current_image)


#noise reduction on grayscaled image
grayscaled_image = grayscale(background_removed_image)
display_image(grayscaled_image)
inverted_grayscaled_image = invert_grayscale(grayscaled_image)
    
#enhance black pixels intensity
blackhat_image = black_hat(inverted_grayscaled_image)
display_image(blackhat_image)

#remove surface reflection points
removed_refection = cv2.add(inverted_grayscaled_image, blackhat_image)
image_without_reflection = median_blur(removed_refection)
display_image(image_without_reflection)

#Edge Enchancement
edged_image = canny_edge(image_without_reflection)
display_image(edged_image)

#fourth step get circular images
circles = hough_circles(edged_image)
if circles is not None:
    #fifth step mark circles co-ordinates
    inner_circle = draw_circle(circles, current_image)
    display_image(current_image)
    x, y,_ = current_image.shape

    #crop iris part from grayscale image
    crop_image(x, y, inner_circle, grayscaled_image)
    display_image(grayscaled_image)
cv2.destroyAllWindows()

